In [1]:
import json
import math
import numpy as np
import pandas as pd

# >>> Pfad anpassen (bei dir wahrscheinlich: "./web/data/btc.json")
BTC_JSON = "btc.json"

TRADING_FEE = 0.0005   # 0.05% pro gehandeltem Portfoliovolumen
BLOCK_LEN   = 45       # Tage pro Block (Bootstrap)
N_SIMS      = 400      # Monte-Carlo Läufe

# -----------------------------
# Backtest Engine
# -----------------------------
def perf_stats(pv: np.ndarray):
    print("perf_stats")
    r = pv[1:] / pv[:-1] - 1.0
    n = len(r)
    years = n / 252.0

    cagr = (pv[-1] / pv[0]) ** (1 / years) - 1 if pv[-1] > 0 else -1.0
    vol = np.std(r, ddof=1) * math.sqrt(252) if np.std(r, ddof=1) > 0 else 0.0
    sharpe = (np.mean(r) / np.std(r, ddof=1)) * math.sqrt(252) if np.std(r, ddof=1) > 0 else 0.0

    peak = -np.inf
    mdd = 0.0
    for v in pv:
        peak = max(peak, v)
        dd = v / peak - 1.0
        mdd = min(mdd, dd)

    return {
        "CAGR": cagr,
        "Vol": vol,
        "Sharpe": sharpe,
        "MaxDD": mdd,
        "Final": pv[-1],
    }

def backtest_ratio_hysteresis(ret, ratio_sig, buy_th, sell_start, gamma, fee=TRADING_FEE):
    """
    Hysterese:
      - ratio <= buy_th     => target = 1.0
      - buy_th < ratio < sell_start => target = prev_w (kein Nachkauf)
      - ratio >= sell_start => target = min(prev_w, w_sell(ratio)) (nur verkaufen)
    """
    n = len(ret)
    pv = np.empty(n + 1, dtype=float)
    pv[0] = 1.0

    prev_w = 1.0
    turnover = 0.0

    width = 100.0 - sell_start
    if width <= 0:
        raise ValueError("sell_start must be < 100")

    for t in range(n):
        r = float(ratio_sig[t])

        if r <= buy_th:
            target = 1.0
        elif r < sell_start:
            target = prev_w
        else:
            x = (r - sell_start) / width
            w_sell = max(0.0, 1.0 - x) ** gamma
            target = min(prev_w, w_sell)

        dw = target - prev_w
        cost = fee * abs(dw) * pv[t]
        pv_after = pv[t] - cost

        pv[t + 1] = pv_after * (1.0 + target * ret[t])

        turnover += abs(dw)
        prev_w = target

    return pv, turnover

# -----------------------------
# Monte Carlo (Block Bootstrap)
# -----------------------------
def block_bootstrap_indices(n, block_len, rng):
    idx = []
    while len(idx) < n:
        start = rng.integers(0, n - block_len + 1)
        idx.extend(range(start, start + block_len))
    return np.array(idx[:n], dtype=int)

def mc_eval(ret, ratio_sig, params_list, n_sims=N_SIMS, block_len=BLOCK_LEN, fee=TRADING_FEE, seed=1):
    rng = np.random.default_rng(seed)
    n = len(ret)

    rows = []
    for sim in range(n_sims):
        idx = block_bootstrap_indices(n, block_len, rng)
        ret_s = ret[idx]
        ratio_s = ratio_sig[idx]

        # Buy & Hold
        pv_bh = np.cumprod(np.r_[1.0, 1.0 + ret_s])
        st_bh = perf_stats(pv_bh)
        rows.append({"sim": sim, "strategy": "buy_hold", **st_bh, "turnover": 0.0})

        # Strategies
        for (buy_th, sell_start, gamma) in params_list:
            pv, to = backtest_ratio_hysteresis(ret_s, ratio_s, buy_th, sell_start, gamma, fee=fee)
            st = perf_stats(pv)
            name = f"buy{buy_th}_sell{sell_start}_g{gamma}"
            rows.append({"sim": sim, "strategy": name, **st, "turnover": to})

    return pd.DataFrame(rows)

def summarize(mc: pd.DataFrame):
    out = []
    for strat, g in mc.groupby("strategy"):
        out.append({
            "strategy": strat,
            "CAGR_med": np.median(g["CAGR"]),
            "CAGR_p10": np.percentile(g["CAGR"], 10),
            "CAGR_p90": np.percentile(g["CAGR"], 90),
            "Vol_med": np.median(g["Vol"]),
            "Sharpe_med": np.median(g["Sharpe"]),
            "MaxDD_med": np.median(g["MaxDD"]),
            "Turnover_med": np.median(g["turnover"]),
        })
    return pd.DataFrame(out).sort_values("Sharpe_med", ascending=False)

# -----------------------------
# Load data + run
# -----------------------------
with open(BTC_JSON, "r") as f:
    data = json.load(f)

price = np.array(data["series"]["price"], dtype=float)
ratio = np.array(data["series"]["ratio"], dtype=float)

# returns t = 1..end, signal uses ratio at t-1 (kein Lookahead innerhalb eines Tages)
ret = price[1:] / price[:-1] - 1.0
ratio_sig = ratio[:-1]

# Grid (du kannst das enger/weiter machen)
buy_th_list = [10, 15, 20, 25]
sell_start_list = [40, 45, 50, 55, 60]
gamma_list = [1, 2, 3, 4, 5]

params = [(b, s, g) for b in buy_th_list for s in sell_start_list for g in gamma_list]

mc = mc_eval(ret, ratio_sig, params, n_sims=N_SIMS, block_len=BLOCK_LEN, fee=TRADING_FEE, seed=2)
summ = summarize(mc)

print("\nTop 15 nach median Sharpe:")
print(summ.head(15).to_string(index=False))

print("\nTop 15 nach niedrigster Volatilität (bei Sharpe>0.8 als Filter):")
lowv = summ[summ["Sharpe_med"] > 0.8].sort_values("Vol_med", ascending=True).head(15)
print(lowv.to_string(index=False))

# Aktueller Ratiospot als Info
latest_ratio = float(ratio[-1])
latest_price = float(price[-1])
latest_date = data["series"]["date"][-1]
print(f"\nLetzter Datenpunkt: {latest_date} | Preis={latest_price:.2f} | Ratio={latest_ratio:.2f}")


perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats
perf_stats